<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [2]:
!wget "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"

--2024-06-24 14:56:43--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: 'concrete_data_week4.zip'

     0K .......... .......... .......... .......... ..........  0% 1.22M 3m25s
    50K .......... .......... .......... .......... ..........  0% 4.44M 2m10s
   100K .......... .......... .......... .......... ..........  0% 4.07M 1m47s
   150K .......... .......... .......... .......... ..........  0% 2.52M 1m45s
   200K .......... .......... .......... .......... ..........  0% 2.55M 1m44s
   250K .......... .......... .......... .......... ..........  0% 2.64M 1m42s
   300

In [5]:
from zipfile import ZipFile

with ZipFile('concrete_data_week4.zip', 'r') as f:
    #extract in current directory
    f.extractall()

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [16]:
num_classes = 2
image_resize = 224
batch_size = 100

In [22]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [24]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize,image_resize),
    batch_size=batch_size,
    class_mode='categorical',)

Found 30001 images belonging to 2 classes.


In [26]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize,image_resize),
    batch_size=batch_size,
    class_mode='categorical',)

Found 9501 images belonging to 2 classes.


In [31]:
model = Sequential()

In [33]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights="imagenet",
))

58889256/58889256 [==============================] - 12s 0us/step


In [35]:
model.add(Dense(
    num_classes,
    activation='softmax'
))

In [39]:
model.layers[0].trainable=False

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [44]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

<a id="item43"></a>

In [50]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

C:\Users\Garrett\AppData\Local\Temp\ipykernel_11684\251737888.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2
301/301 [==============================] - 1745s 6s/step - loss: 0.1888 - accuracy: 0.9286 - val_loss: 0.0371 - val_accuracy: 0.9917
Epoch 2/2
301/301 [==============================] - 1783s 6s/step - loss: 0.0274 - accuracy: 0.9947 - val_loss: 0.0209 - val_accuracy: 0.9946


## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [54]:
model_resnet = keras.saving.load_model('classifier_resnet_model.h5')

In [56]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize,image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [58]:
results = model.evaluate_generator(
    test_generator,
    verbose=1)

C:\Users\Garrett\AppData\Local\Temp\ipykernel_11684\3649748436.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  results = model.evaluate_generator(


16/16 [==============================] - 24s 1s/step - loss: 0.0192 - accuracy: 0.9960


In [64]:
results_resnet = model_resnet.evaluate_generator(
    test_generator,
    verbose=1)

C:\Users\Garrett\AppData\Local\Temp\ipykernel_11684\691439979.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  results_resnet = model_resnet.evaluate_generator(


16/16 [==============================] - 19s 1s/step - loss: 0.0023 - accuracy: 1.0000


In [66]:
print(f'VGG16 Loss: {results[0]}')
print(f'VGG16 Acc: {results[1]}')
print(f'ResNet Loss: {results_resnet[0]}')
print(f'ResNet Acc: {results_resnet[1]}')

VGG16 Loss: 0.019234422594308853
VGG16 Acc: 0.9959999918937683
ResNet Loss: 0.0022780161816626787
ResNet Acc: 1.0


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [78]:
vgg_pred = model.predict_generator(
    test_generator,
    verbose=1,)

C:\Users\Garrett\AppData\Local\Temp\ipykernel_11684\1880822431.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  vgg_pred = model.predict_generator(


16/16 [==============================] - 23s 1s/step


In [98]:
y_classes = vgg_pred.argmax(axis=-1)[0:5]
for i,y in enumerate(y_classes):
    if y == 0:
        print(f'{i}:Negative')
    else:
        print(f'{i}:Positive')

0:Negative
1:Negative
2:Negative
3:Negative
4:Negative


In [100]:
resNet_pred = model.predict_generator(
    test_generator,
    verbose=1,)

C:\Users\Garrett\AppData\Local\Temp\ipykernel_11684\1675929221.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  resNet_pred = model.predict_generator(


16/16 [==============================] - 23s 1s/step


In [102]:
y_classes_resnet = resNet_pred.argmax(axis=-1)[0:5]
for i,y in enumerate(y_classes):
    if y == 0:
        print(f'{i}:Negative')
    else:
        print(f'{i}:Positive')

0:Negative
1:Negative
2:Negative
3:Negative
4:Negative


In [106]:
y_classes = test_generator.classes
y_classes_vgg = vgg_pred.argmax(axis=-1)
y_classes_resnet = resNet_pred.argmax(axis=-1)

# Lists to store incorrect results
incorrect_vgg = []
incorrect_resnet = []

# Iterate over the lists
for i in range(len(y_classes)):
    test_val = y_classes[i]
    vgg_result = y_classes_vgg[i]
    resnet_result = y_classes_resnet[i]
    
    # Check correctness for VGG
    if vgg_result != test_val:
        incorrect_vgg.append((i, vgg_result))  # Store index and incorrect value
    
    # Check correctness for ResNet
    if resnet_result != test_val:
        incorrect_resnet.append((i, resnet_result))  # Store index and incorrect value

# Print results
print("\nIncorrect VGG results (index, value):")
for idx, value in incorrect_vgg:
    print(f"Index {idx}: {value}")

print("\nIncorrect ResNet results (index, value):")
for idx, value in incorrect_resnet:
    print(f"Index {idx}: {value}")


Incorrect VGG results (index, value):
Index 342: 0
Index 441: 0

Incorrect ResNet results (index, value):
Index 342: 0
Index 441: 0


In [108]:
print(f'VGG Accuracy: {1- len(incorrect_vgg)/len(y_classes_vgg)}')
print(f'Resnet Accuracy: {1- len(incorrect_resnet)/len(y_classes_resnet)}')

VGG Accuracy: 0.996
Resnet Accuracy: 0.996


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).